In [1]:
import cvxpy
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
sample = pd.read_csv('sample.csv')

In [3]:
sample.head(3)

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [4]:
# クライアトa, b, cの元々持っていた枠をランダムに決める。
length = len(sample)
ids = list(range(length))
# idは1からなので、1を足す。
random_ids = [id+1 for id in ids]

# Break Point 1の略。
bp1 = length//3 
bp2 = length//3 + bp1

random.shuffle(random_ids)
a, b, c = random_ids[:bp1], random_ids[bp1:bp2], random_ids[bp2:]

In [5]:
# それぞれのidを表示
print(a)
print(b)
print(c)

[11, 16, 30, 1, 8, 15, 29, 23, 12, 17]
[2, 22, 19, 13, 18, 10, 5, 24, 28, 7]
[25, 3, 20, 27, 9, 6, 26, 21, 14, 4]


In [6]:
# それぞれのidのデータのみを取り出すための判別用カラム。
sample['judge_a'] = sample['id'].apply(lambda x:x in a)
sample['judge_b'] = sample['id'].apply(lambda x:x in b)
sample['judge_c'] = sample['id'].apply(lambda x:x in c)

In [7]:
# それぞれのidのデータを取得。
df_a = sample[sample['judge_a']].loc[:, ['id','price_A', 'rating_A']]
df_b = sample[sample['judge_b']].loc[:, ['id','price_B', 'rating_B']]
df_c = sample[sample['judge_c']].loc[:, ['id','price_C', 'rating_C']]

In [8]:
# 判別用カラムを削除。
del sample['judge_a']
del sample['judge_b']
del sample['judge_c']

In [9]:
# 元々の値段の総和、元々の視聴率の和を計算する。
# Original price, Original point
o_price_a, o_point_a = sum(df_a['price_A']), sum(df_a['rating_A'])
o_price_b, o_point_b = sum(df_b['price_B']), sum(df_b['rating_B'])
o_price_c, o_point_c = sum(df_c['price_C']), sum(df_c['rating_C'])

In [10]:
# 獲得した枠のidを貯めておく。
obtained_id_A = []
obtained_id_B = []
obtained_id_C = []

In [11]:
# 獲得した枠の価格を貯めていく。
obtained_price_A = 0
obtained_price_B = 0
obtained_price_C = 0

In [12]:
# 獲得した枠の視聴率を貯めていく。
obtained_point_A = 0
obtained_point_B = 0
obtained_point_C = 0

In [108]:
# ここに結果を格納していき、何度も繰り返す。（視聴率和の合計値が'key'で、その時のidが'value'）
result = dict()

In [13]:
# 各クライアントの考える価値を数値化し、表に加える。
# 指定した視聴率/そのクライアントの号数価格
sample['value_A'] = round((sample['rating_A']/sample['price_A'])*1000, 2)
sample['value_B'] = round((sample['rating_B']/sample['price_B'])*1000, 2)
sample['value_C'] = round((sample['rating_C']/sample['price_C'])*1000, 2)

In [14]:
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)
sample.head(3)

client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,2,943,947,961,10.1,8.9,9.3,10.71,9.40,9.68
2,3,980,980,987,10.4,8.7,10.3,10.61,8.88,10.44


<b><font color='Red'>※valueは、基本的に正の数じゃないと、選ばなくなってしまう（０の方が大きいから。）</font></b>

***
***

### 〜ここからアルゴリズム〜

In [15]:
# ナップザック問題。

# size = 値段のデータ。
# weight = 価値のデータ。
# capacity = 元々の値段×1.05倍の値段。
# ids = 取得すべきだと考えられた枠のid    

def Knapsack(size, weight, capacity, obtained=0, rate=0.05):
    # 価格が元の価格の1.05%を超えても、終わり。
    if capacity*(1 + rate) - obtained > 0:        
        x = cvxpy.Variable(size.shape[0], boolean=True)
        # 要素の個数(これを加えて平均を最大化しないと、価格の小さいものばかりをとってしまう。)
        count = sum(np.ones(len(size)) * x)
        # 目的
        objective = cvxpy.Maximize(weight * x / count)
        # 制限。ここでは、capacity(価格)の上下5%を取っている。
        constraints = [capacity*(1 + rate) -  obtained >= size * x]
        constraints += [capacity*(1 - rate) - obtained <= size * x]

        prob = cvxpy.Problem(objective, constraints)
        prob.solve(solver=cvxpy.ECOS_BB)
        result = [round(ix, 0) for ix in x.value]

        return result
    else:
        return []

In [16]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

if obtained_point_A < o_point_a:
    # capacityの値を制約しすぎると、その値の枠に収まらなくなる可能性がある。
    # その場合は、±の幅(rate)を大きくすれば良い。
    ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=0.1)
else:
    ids_a = [0 for i in range(len(size_A))]

In [17]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

if obtained_point_B < o_point_b:
    ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)
else:
    ids_b = [0 for i in range(len(size_B))]

In [18]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

if obtained_point_C < o_point_c:
    ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)
else:
    ids_c = [0 for i in range(len(size_C))]

In [19]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [20]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

6.0 個の枠: [-1.  1.  2. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 13. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 27. 28. -1.]
6.0 個の枠: [-1. -1.  2. -1.  4. -1. -1. -1. -1. -1. -1. -1. -1. 13. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 27. 28. -1.]
6.0 個の枠: [-1.  1.  2. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 13. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 27. 28. -1.]


In [21]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [22]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [23]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [1, 2, 16, 28]
client_b: [4]
client_c: [13, 27]


In [24]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [25]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [26]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [27]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [28]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

残りの枠: 23
client_Aの合計価格: 4253 client_Aの合計視聴率 44.1
client_Bの合計価格: 149 client_Bの合計視聴率 1.5
client_Cの合計価格: 2325 client_Cの合計視聴率 23.8


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55


client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


In [29]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

### アルゴリズム２回目

In [30]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

if obtained_point_A < o_point_a:
    ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)
else:
    ids_a = [0 for i in range(len(size_A))]

In [31]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

if obtained_point_B < o_point_b:
    ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=0.1)
else:
    ids_b = [0 for i in range(len(size_B))]

In [32]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

if obtained_point_C < o_point_c:
    ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)
else:
    ids_c = [0 for i in range(len(size_C))]

In [33]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [34]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

3.0 個の枠: [-1. -1. -1. -1. -1.  5. -1. -1.  8. -1. -1. -1. -1. -1. -1. 15. -1. -1.
 -1. -1. -1. -1. -1.]
8.0 個の枠: [-1. -1. -1.  3.  4.  5. -1.  7.  8. -1. -1. -1. -1. 13. -1. 15. -1. -1.
 -1. -1. 20. -1. -1.]
5.0 個の枠: [-1. -1. -1.  3.  4.  5. -1. -1.  8. -1. -1. -1. -1. -1. -1. 15. -1. -1.
 -1. -1. -1. -1. -1.]


In [35]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [36]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [37]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: [3, 5, 7, 8, 13, 20]
client_c: [4, 15]


In [38]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [39]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [40]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [41]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [42]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

残りの枠: 15
client_Aの合計価格: 4253 client_Aの合計視聴率 44.1
client_Bの合計価格: 4548 client_Bの合計視聴率 46.800000000000004
client_Cの合計価格: 4014 client_Cの合計視聴率 41.6


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55


client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


In [43]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

### アルゴリズム３回目

In [44]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

if obtained_point_A < o_point_a:
    ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)
else:
    ids_a = [0 for i in range(len(size_A))]

In [45]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

if obtained_point_B < o_point_b:
    ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)
else:
    ids_b = [0 for i in range(len(size_B))]

In [46]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

if obtained_point_C < o_point_c:
    ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)
else:
    ids_c = [0 for i in range(len(size_C))]

In [47]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [48]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

4.0 個の枠: [-1. -1. -1. -1.  4. -1. -1. -1. -1. -1. -1. 11. 12. -1. 14.]
2.0 個の枠: [-1. -1. -1. -1.  4. -1. -1. -1. -1. -1. -1. 11. -1. -1. -1.]
4.0 個の枠: [-1. -1. -1. -1.  4. -1. -1. -1. -1. -1. -1. 11. 12. -1. 14.]


In [49]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [50]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [51]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: [4, 11]
client_c: [12, 14]


In [52]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [53]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [54]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [55]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [56]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

残りの枠: 11
client_Aの合計価格: 4253 client_Aの合計視聴率 44.1
client_Bの合計価格: 5808 client_Bの合計視聴率 59.60000000000001
client_Cの合計価格: 5212 client_Cの合計視聴率 53.2


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55


client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


In [57]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

### アルゴリズム４回目

In [58]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

if obtained_point_A < o_point_a:
    ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)
else:
    ids_a = [0 for i in range(len(size_A))]

In [59]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

if obtained_point_B < o_point_b:
    ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)
else:
    ids_b = [0 for i in range(len(size_B))]

In [60]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

if obtained_point_C < o_point_c:
    ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)
else:
    ids_c = [0 for i in range(len(size_C))]

In [61]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [62]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

6.0 個の枠: [ 0. -1. -1.  3. -1. -1.  6.  7. -1.  9. 10.]
0.0 個の枠: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
3.0 個の枠: [-1. -1. -1. -1. -1. -1.  6.  7. -1. -1. 10.]


In [63]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [64]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [65]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [0, 3, 9]
client_b: []
client_c: [6, 7, 10]


In [66]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [67]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [68]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [69]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [70]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

残りの枠: 5
client_Aの合計価格: 5343 client_Aの合計視聴率 55.7
client_Bの合計価格: 5808 client_Bの合計視聴率 59.60000000000001
client_Cの合計価格: 6631 client_Cの合計視聴率 66.0


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
1,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55
2,15,230,212,218,2.4,2.0,2.5,10.43,9.43,11.47


client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


In [71]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

### アルゴリズム５回目

In [72]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

if obtained_point_A < o_point_a:
    ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)
else:
    ids_a = [0 for i in range(len(size_A))]

In [73]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

if obtained_point_B < o_point_b:
    ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)
else:
    ids_b = [0 for i in range(len(size_B))]

In [74]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

if obtained_point_C < o_point_c:
    ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)
else:
    ids_c = [0 for i in range(len(size_C))]

In [75]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [76]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

4.0 個の枠: [ 0. -1.  2.  3.  4.]
0.0 個の枠: [-1. -1. -1. -1. -1.]
0 個の枠: [-1 -1 -1 -1 -1]


In [77]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [78]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [79]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [0, 2, 3, 4]
client_b: []
client_c: []


In [80]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [81]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [82]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [83]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [84]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

残りの枠: 1
client_Aの合計価格: 6410 client_Aの合計視聴率 66.0
client_Bの合計価格: 5808 client_Bの合計視聴率 59.60000000000001
client_Cの合計価格: 6631 client_Cの合計視聴率 66.0


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55


client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


In [85]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

### アルゴリズム６回目

In [86]:
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = o_price_a

if obtained_point_A < o_point_a:
    ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A)
else:
    ids_a = [0 for i in range(len(size_A))]

In [87]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = o_price_b

if obtained_point_B < o_point_b:
    ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B)
else:
    ids_b = [0 for i in range(len(size_B))]

In [88]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = o_price_c

if obtained_point_C < o_point_c:
    ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C)
else:
    ids_c = [0 for i in range(len(size_C))]

In [89]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [90]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

0 個の枠: [-1 -1]
1.0 個の枠: [ 0. -1.]
0 個の枠: [-1 -1]


In [91]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [92]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [93]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: [0]
client_c: []


In [94]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [95]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [96]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [97]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [98]:
print('残りの枠:', len(sample))
print('client_Aの合計価格:', obtained_price_A, 'client_Aの合計視聴率', obtained_point_A)
print('client_Bの合計価格:', obtained_price_B, 'client_Bの合計視聴率', obtained_point_B)
print('client_Cの合計価格:', obtained_price_C, 'client_Cの合計視聴率', obtained_point_C)
display(sample.head(3))
print('client_Aの交換前の合計価格:', o_price_a, 'client_Aの交換前の合計視聴率', o_point_a)
print('client_Bの交換前の合計価格:', o_price_b, 'client_Bの交換前の合計視聴率', o_point_b)
print('client_Cの交換前の合計価格:', o_price_c, 'client_Cの交換前の合計視聴率', o_point_c)

残りの枠: 1
client_Aの合計価格: 6410 client_Aの合計視聴率 66.0
client_Bの合計価格: 5963.0 client_Bの合計視聴率 61.00000000000001
client_Cの合計価格: 6631 client_Cの合計視聴率 66.0


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


client_Aの交換前の合計価格: 6520 client_Aの交換前の合計視聴率 65.0
client_Bの交換前の合計価格: 6073 client_Bの交換前の合計視聴率 59.8
client_Cの交換前の合計価格: 6417 client_Cの交換前の合計視聴率 64.39999999999999


In [99]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

finish!!!!


***
***
***

In [126]:
if obtained_point_A >o_point_a and obtained_point_B >o_point_b and obtained_point_C >o_point_c:
    print('Good Flame turn!!!')
    sum_point = obtained_point_A + obtained_point_B + obtained_point_C 
    result[sum_point] = [obtained_id_A, obtained_id_B, obtained_id_C]

Good Flame turn!!!


In [119]:
result

{193.0: [[1, 2, 16, 28, 0, 3, 9, 0, 2, 3, 4],
  [4, 3, 5, 7, 8, 13, 20, 4, 11, 0],
  [13, 27, 4, 15, 12, 14, 6, 7, 10]]}

### これを何度も繰り返すことにより、resultに値を格納していく。

In [125]:
# 最も視聴率和を大きくした交換の仕方を求める。
result[max(result.keys())]

[[1, 2, 16, 28, 0, 3, 9, 0, 2, 3, 4],
 [4, 3, 5, 7, 8, 13, 20, 4, 11, 0],
 [13, 27, 4, 15, 12, 14, 6, 7, 10]]

### プロトタイプは完成。あとはエラーの処理をし、アルゴリズム化する。